In [47]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import math
import re

import random
import time

### 1 Читаем файл с названиями фильмов

In [2]:
# Чтение файла данных о фильмах (item_data)
file_path = r'../ml-100k\u.item'

column_names = ['item_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_url', 
               'unknown', 'action', 'adventure', 'animation', 'children', 'comedy', 'crime', 
               'documentary', 'drama', 'fantasy', 'film_noir', 'horror', 'musical', 'mystery', 
               'romance', 'sci_fi', 'thriller', 'war', 'western']

# Чтение данных, указывая, что разделитель — табуляция
movie_data = pd.read_csv(file_path, sep='|', names=column_names, encoding='ISO-8859-1')

In [3]:
movie_data.head()

,item_id,movie_title,release_date,video_release_date,IMDb_url,unknown,action,adventure,animation,children,...,fantasy,film_noir,horror,musical,mystery,romance,sci_fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
movie_data['movie_title'].iloc[0]

'Toy Story (1995)'

#### Эта ссылка нерабочая

In [5]:
movie_data['IMDb_url'].iloc[0]

'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)'

### 2 Ищем нужные поля в большой json файле

У нас нет конкретных ссылок на фильм, нам нужно  их добыть по названию.  
На сайте Imdb есть поисковая строка и мы туда вбиваем название фильма, например, "Toy Story" и он выдает много различных вариантов.  

Вот пример такого  поиска:  
https://www.imdb.com/find/?q=toy%20story&ref_=nv_sr_sm  


Таким образом нам нужно получить все ссылки на страницы эти фильмов, который появились в результате поиска "Toy Story", чтоыб потом выбрать нужный для нас фильм.  
Далее мы можем выбрать именно нужный нам фильм по какому-то придуманному нами критерию, например, фильм с самым большим рейтингов или числом голосов или по году релиза (но год есть не у всех фильмов)  

#### Пример для одного фильма "Toy story", чтобы получить список фильмов с таким названием

#### Записываем в result 4 блока данных:  

 - "title" - название фильма  
 - "url" - ссылка на страницу фильма   
 - "type" - различная информация связанная с фильмом (можно и не собирать эту инфу)  
 - "year" - год выпуска  
 
 ** Самое главное что мы собираем ссылки на страницы конкретных фильмов  

In [6]:
import requests
from bs4 import BeautifulSoup
import json

query = "Toy Story 1995"
url = f"https://www.imdb.com/find/?q={query.replace(' ', '+')}"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Находим все результаты поиска
results = []
for item in soup.select('li.ipc-metadata-list-summary-item'):
    title = item.select_one('a.ipc-metadata-list-summary-item__t')
    if not title:
        continue
    
    result = {
        "title": title.text,
        "url": "https://www.imdb.com" + title["href"],
        "type": item.select_one('div.ipc-metadata-list-summary-item__c').text if item.select_one('div.ipc-metadata-list-summary-item__c') else None,
        "year": item.select_one('span.ipc-metadata-list-summary-item__li').text if item.select_one('span.ipc-metadata-list-summary-item__li') else None,
    }
    results.append(result)

print(json.dumps(results, indent=4, ensure_ascii=False))

[
    {
        "title": "Toy Story",
        "url": "https://www.imdb.com/title/tt0114709/?ref_=fn_all_ttl_1",
        "type": "Toy Story1995Tom Hanks, Tim Allen",
        "year": "1995"
    },
    {
        "title": "Toy Story",
        "url": "https://www.imdb.com/title/tt0178952/?ref_=fn_all_ttl_2",
        "type": "Toy Story1995Video GameCorey Burton, R. Lee Ermey",
        "year": "1995"
    },
    {
        "title": "To Infinity and Beyond: The Making of 'Toy Story'",
        "url": "https://www.imdb.com/title/tt0245255/?ref_=fn_all_ttl_3",
        "type": "To Infinity and Beyond: The Making of 'Toy Story'1995TV ShortAnnie Potts, Wil Shriner",
        "year": "1995"
    },
    {
        "title": "Toy Story Sega Genesis and Super Nintendo Commercial",
        "url": "https://www.imdb.com/title/tt32380999/?ref_=fn_all_ttl_4",
        "type": "Toy Story Sega Genesis and Super Nintendo Commercial1995Video",
        "year": "1995"
    },
    {
        "title": "Toy Story 4 Trailer 19

### 3 Находим нужные поля для нашего фильма

Берем как пример 1-ый фильм из списка **results** и попробуем найти нужные по нему поля. Так как все фильмы имеют похожую структуру дерева(json) полей, поэтому нам нужно найти ключи как достучаться до нужных полей

In [7]:
url = results[0]["url"]
print(url)

https://www.imdb.com/title/tt0114709/?ref_=fn_all_ttl_1


In [8]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

In [9]:
type(soup)

bs4.BeautifulSoup

**full_data** - это код страницы конкретного фильма, где мы можем найти путь как обратиться к любой информации со страницы

In [10]:
data_tag = soup.find("script", id="__NEXT_DATA__", type="application/json")
full_data = json.loads(data_tag.string)

In [11]:
full_data

{'props': {'pageProps': {'tconst': 'tt0114709',
   'aboveTheFoldData': {'id': 'tt0114709',
    'productionStatus': {'currentProductionStage': {'id': 'released',
      'text': 'Released',
      '__typename': 'ProductionStage'},
     'productionStatusHistory': [{'status': {'id': 'released',
        'text': 'Released',
        '__typename': 'ProductionStatus'},
       '__typename': 'ProductionStatusHistory'}],
     'restriction': None,
     '__typename': 'ProductionStatusDetails'},
    'canHaveEpisodes': False,
    'series': None,
    'titleText': {'text': 'Toy Story', '__typename': 'TitleText'},
    'titleType': {'displayableProperty': {'value': {'plainText': '',
       '__typename': 'Markdown'},
      '__typename': 'DisplayableTitleTypeProperty'},
     'text': 'Movie',
     'id': 'movie',
     'isSeries': False,
     'isEpisode': False,
     'categories': [{'value': 'movie', '__typename': 'TitleTypeCategory'}],
     'canHaveEpisodes': False,
     '__typename': 'TitleType'},
    'origina

In [12]:
full_data.keys()

dict_keys(['props', 'page', 'query', 'buildId', 'assetPrefix', 'runtimeConfig', 'isFallback', 'dynamicIds', 'gssp', 'locale', 'locales', 'defaultLocale', 'scriptLoader'])

#### Делаем функцию, которой подаем на вход большое дерево(код) страницы full_data и то что мы хотим найти, например, Tom Hanks, чтобы найти пути как до него добраться по дереву.  
**Tom Hanks** - мы находим своими глазами, эта именна та информация которую мы посчитали интересной, чтобы спарсить

In [13]:
def find_value(data, target, path=None, results=None):
    if path is None:
        path = []
    if results is None:
        results = []

    if isinstance(data, dict):
        for key, value in data.items():
            find_value(value, target, path + [f"[{repr(key)}]"], results)
    elif isinstance(data, list):
        for idx, item in enumerate(data):
            find_value(item, target, path + [f"[{idx}]"], results)
    else:
        if data == target:
            results.append("".join(path))

    return results

### ID

Мы видим в ссылке на фильм что у него ID "tt0114709"   
Давай найдем этот ID, где он прячется в коде страницы фильма

In [14]:
paths = find_value(full_data, "tt0114709")

for p in paths:
    print(f"Нашёл по пути: data{p}")

Нашёл по пути: data['props']['pageProps']['tconst']
Нашёл по пути: data['props']['pageProps']['aboveTheFoldData']['id']
Нашёл по пути: data['props']['pageProps']['aboveTheFoldData']['meta']['canonicalId']
Нашёл по пути: data['props']['pageProps']['mainColumnData']['id']
Нашёл по пути: data['props']['pageProps']['requestContext']['pageConst']
Нашёл по пути: data['props']['pageProps']['requestContext']['queryParams']['tconst']
Нашёл по пути: data['query']['tconst']


Можно выбрать лубую ссылку и потом уже в ходе работы посмотреть, какая из них работает хорошо на 100 фильмах, то есть находит ли все ссылки или много пустых значений

### Director

In [15]:
paths = find_value(full_data, "John Lasseter")

for p in paths:
    print(f"Нашёл по пути: data{p}")

Нашёл по пути: data['props']['pageProps']['aboveTheFoldData']['principalCredits'][0]['credits'][0]['name']['nameText']['text']
Нашёл по пути: data['props']['pageProps']['aboveTheFoldData']['principalCredits'][1]['credits'][0]['name']['nameText']['text']
Нашёл по пути: data['props']['pageProps']['aboveTheFoldData']['directorsPageTitle'][0]['credits'][0]['name']['nameText']['text']
Нашёл по пути: data['props']['pageProps']['mainColumnData']['directors'][0]['credits'][0]['name']['nameText']['text']
Нашёл по пути: data['props']['pageProps']['mainColumnData']['writers'][0]['credits'][0]['name']['nameText']['text']


### Rating

In [16]:
paths = find_value(full_data, 8.3)

for p in paths:
    print(f"Нашёл по пути: data{p}")

Нашёл по пути: data['props']['pageProps']['aboveTheFoldData']['ratingsSummary']['aggregateRating']
Нашёл по пути: data['props']['pageProps']['mainColumnData']['moreLikeThisTitles']['edges'][0]['node']['ratingsSummary']['aggregateRating']
Нашёл по пути: data['props']['pageProps']['mainColumnData']['moreLikeThisTitles']['edges'][4]['node']['ratingsSummary']['aggregateRating']


### voteCount

как я нашел это число ? **1118705**  
а просто у меня же есть full_data и из сайта глазами я нашел первые 4 числа "1118" и через ctrl + F нашел это число

In [17]:
paths = find_value(full_data, 1118705)

for p in paths:
    print(f"Нашёл по пути: data{p}")

### Stars

In [18]:
paths = find_value(full_data, "Tom Hanks")

for p in paths:
    print(f"Нашёл по пути: data{p}")

Нашёл по пути: data['props']['pageProps']['aboveTheFoldData']['principalCredits'][2]['credits'][0]['name']['nameText']['text']
Нашёл по пути: data['props']['pageProps']['aboveTheFoldData']['castPageTitle']['edges'][0]['node']['name']['nameText']['text']
Нашёл по пути: data['props']['pageProps']['mainColumnData']['cast']['edges'][0]['node']['name']['nameText']['text']


### Budget

In [19]:
paths = find_value(full_data, 30000000)

for p in paths:
    print(f"Нашёл по пути: data{p}")

Нашёл по пути: data['props']['pageProps']['mainColumnData']['productionBudget']['budget']['amount']


### Gross worldwide

In [20]:
paths = find_value(full_data, 394436586)

for p in paths:
    print(f"Нашёл по пути: data{p}")

Нашёл по пути: data['props']['pageProps']['mainColumnData']['worldwideGross']['total']['amount']


### Opening weekend US & Canada

In [21]:
paths = find_value(full_data, 1995)

for p in paths:
    print(f"Нашёл по пути: data{p}")

Нашёл по пути: data['props']['pageProps']['aboveTheFoldData']['releaseYear']['year']
Нашёл по пути: data['props']['pageProps']['mainColumnData']['releaseYear']['year']
Нашёл по пути: data['props']['pageProps']['mainColumnData']['connections']['edges'][0]['node']['associatedTitle']['releaseYear']['year']


In [22]:
print(full_data['props']['pageProps']['mainColumnData']['releaseDate']['day'])
print(full_data['props']['pageProps']['mainColumnData']['releaseDate']['month'])
print(full_data['props']['pageProps']['mainColumnData']['releaseDate']['year'])

21
3
1996


### Runtime

Как нашел ? "1h 21m"  
Вбил через ctrl + F 21 и увидел это в большом дереве страницы  

так же 1h 21m это 4860 секунд, поэтому решил вбить 48 и найти время по скундам

In [23]:
paths = find_value(full_data, "1h 21m")

for p in paths:
    print(f"Нашёл по пути: data{p}")

Нашёл по пути: data['props']['pageProps']['aboveTheFoldData']['runtime']['displayableProperty']['value']['plainText']


In [24]:
paths = find_value(full_data, 4860)

for p in paths:
    print(f"Нашёл по пути: data{p}")

Нашёл по пути: data['props']['pageProps']['aboveTheFoldData']['runtime']['seconds']
Нашёл по пути: data['props']['pageProps']['mainColumnData']['runtime']['seconds']


#### В качестве примера можно посмотреть как будет выглядеть информация

In [25]:
[
    # ID
    full_data['props']['pageProps']['tconst'],
    
    # Director
    full_data['props']['pageProps']['aboveTheFoldData']['principalCredits'][0]['credits'][0]['name']['nameText']['text'],
    
    # Rating
    full_data['props']['pageProps']['aboveTheFoldData']['ratingsSummary']['aggregateRating'],
    
    # voteCount
    full_data['props']['pageProps']['aboveTheFoldData']['ratingsSummary']['voteCount'],
    
    # Stars
    full_data['props']['pageProps']['aboveTheFoldData']['principalCredits'][2]['credits'][0]['name']['nameText']['text'],
    
    # Budget
    full_data['props']['pageProps']['mainColumnData']['productionBudget']['budget']['amount'],
    
    # Gross worldwide
    full_data['props']['pageProps']['mainColumnData']['worldwideGross']['total']['amount'],
    
    # Runtime
    full_data['props']['pageProps']['aboveTheFoldData']['runtime']['seconds'],
    
    
    # Opening weekend US & Canada
    full_data['props']['pageProps']['mainColumnData']['releaseDate']['day'],
    full_data['props']['pageProps']['mainColumnData']['releaseDate']['month'],
    full_data['props']['pageProps']['mainColumnData']['releaseDate']['year']
]

['tt0114709',
 'John Lasseter',
 8.3,
 1118780,
 'Tom Hanks',
 30000000,
 394436586,
 4860,
 21,
 3,
 1996]

### Еще один способ смотреть информацию прямо с сайта, (можно смотреть прямо путь как добираемся до нужной информации)

Для проверки на примере страницы "https://www.imdb.com/title/tt7457314/?ref_=fn_all_ttl_5"  

1) Делаем - Ctrl+Shift+I, открывается справа код страницы, выбираем вверху вкладку **Elements**  
2) Через ctrl+F (поиск) ищем слово "json"
3) 6-ой из 7 json начинается на "{"props":{"pageProps":{"tconst":"tt7457314","aboveTheFoldData":{"id":"tt7457314","produ"  
Правой кнопкой мыши, копируем этот желтый текст
4) Вставляем его сюда в лекое окошко- https://jsonformatter.org/json-viewer   
    
и в правом окошке мы вбиваем в поиске "Sean" видим - что он находится и мы видем где он находится по какому маршруту,  
но почему-то не получилось найти - "Sean Bean", хотя он есть в этом json файле  

Таким образом можно  найти любую инфу то что отображается на странице, например, сравнивая с другими страницами видно что нет **budget**, но его нет и на странице самой и в json файле, поэтому в таблице будет пусто в которую я парсю это значение

### 4 Сбор инорфмации по всем фильмам

In [26]:
from datetime import datetime

Функция, чтобы если у нас далее идет тупик по дереву и не можем добираться до информации не выдавало ошибку

In [27]:
def safe_get(data, keys, default=None):
    for key in keys:
        if isinstance(data, dict):
            data = data.get(key, default)
        elif isinstance(data, list):
            if isinstance(key, int) and 0 <= key < len(data):
                data = data[key]
            else:
                return default
        else:
            return default
    return data

In [28]:
def extract_data(full_data):
    props = safe_get(full_data, ['props', 'pageProps'], {})

    # ID
    tconst = safe_get(props, ['tconst'])
    
    # Director
    director = None
    principal_credits = safe_get(props, ['aboveTheFoldData', 'principalCredits'], [])
    if len(principal_credits) > 0:
        director = safe_get(principal_credits[0], ['credits', 0, 'name', 'nameText', 'text'])

    # Rating & vote count
    rating = safe_get(props, ['aboveTheFoldData', 'ratingsSummary', 'aggregateRating'])
    vote_count = safe_get(props, ['aboveTheFoldData', 'ratingsSummary', 'voteCount'])

    # Stars
    stars = None
    stars = safe_get(principal_credits, [2, 'credits', 0, 'name', 'nameText', 'text'])
    if stars is None:
        stars = safe_get(principal_credits, [2, 'credits', 0, 'name', 'nameText', 'text'])
        
    # Budget & Gross
    budget = safe_get(props, ['mainColumnData', 'productionBudget', 'budget', 'amount'])
    worldwide_gross = safe_get(props, ['mainColumnData', 'worldwideGross', 'total', 'amount'])
    
    # Runtime (in seconds, optional conversion to minutes below)
    runtime = safe_get(props, ['aboveTheFoldData', 'runtime', 'seconds'])
    
    # Release date
    release_day = safe_get(props, ['mainColumnData', 'releaseDate', 'day'])
    release_month = safe_get(props, ['mainColumnData', 'releaseDate', 'month'])
    release_year = safe_get(props, ['mainColumnData', 'releaseDate', 'year'])

    return [
        tconst,
        director,
        rating,
        vote_count,
        stars,
        budget,
        worldwide_gross,
        runtime,
        release_day,
        release_month,
        release_year
    ]


### Проходим по всем фильмам

Отдельно выделим название фильма и дату

In [29]:
print(movie_data['movie_title'].iloc[3])
print(movie_data['movie_title'].iloc[3][:-7])
print(int(movie_data['movie_title'].iloc[3][-5:-1]))

Get Shorty (1995)
Get Shorty
1995


In [30]:
print(movie_data.shape)
movie_data

(1682, 24)


,item_id,movie_title,release_date,video_release_date,IMDb_url,unknown,action,adventure,animation,children,...,fantasy,film_noir,horror,musical,mystery,romance,sci_fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Очень важная функция, чтобы сайт не ругался, что мы скачиваем много  инфы, мы будем рандомно менять **headers**, как будто мы разные пользователи

Когда ты отправляешь запрос с помощью requests.get() (или post()), ты как бы «представляешься» серверу. Заголовки (headers) — это способ передать дополнительную информацию о себе: кто ты, какой у тебя браузер, какие данные ты принимаешь и т.п.

In [99]:
import random

def random_user_agent():
    """Генерирует случайный User-Agent с поддержкой различных браузеров и платформ"""
    # Версии браузеров
    chrome_versions = [
        f"Chrome/{random.randint(120, 122)}.0.{random.randint(6100, 6300)}.{random.randint(80, 99)}",
        f"Chrome/{random.randint(100, 119)}.0.{random.randint(5000, 6000)}.{random.randint(50, 80)}"
    ]
    
    firefox_versions = [
        f"Firefox/{random.randint(115, 120)}.0",
        f"Firefox/{random.randint(100, 114)}.0"
    ]
    
    edge_versions = [
        f"Edg/{random.randint(120, 121)}.0.{random.randint(2100, 2300)}.{random.randint(100, 200)}",
        f"Edg/{random.randint(100, 119)}.0.{random.randint(1500, 2000)}.{random.randint(50, 100)}"
    ]
    
    safari_versions = [
        f"Version/{random.randint(15, 17)}.{random.randint(0, 5)}",
        f"Version/{random.randint(10, 14)}.{random.randint(0, 10)}"
    ]
    
    # Платформы
    platforms = [
        "(Windows NT 10.0; Win64; x64)",
        "(Windows NT 11.0; Win64; x64)",
        "(Macintosh; Intel Mac OS X 10_15_7)",
        "(Macintosh; Intel Mac OS X 14_3_1)",
        "(X11; Linux x86_64)",
        "(X11; Ubuntu; Linux x86_64)"
    ]
    
    # Движки
    engines = [
        "AppleWebKit/537.36 (KHTML, like Gecko)",
        "AppleWebKit/605.1.15 (KHTML, like Gecko)",
        "Gecko/20100101"
    ]
    
    # Собираем случайный User-Agent
    platform = random.choice(platforms)
    engine = random.choice(engines)
    
    browser_type = random.choice(["chrome", "firefox", "edge"])
    
    if browser_type == "chrome":
        browser = random.choice(chrome_versions)
        safari = f"Safari/537.36"
        return f"Mozilla/5.0 {platform} {engine} {browser} {safari}"
    
    elif browser_type == "firefox":
        browser = random.choice(firefox_versions)
        return f"Mozilla/5.0 {platform} {engine} {browser}"
    
    else:  # edge
        browser = random.choice(edge_versions)
        safari = f"Safari/537.36"
        chrome = random.choice(chrome_versions)
        return f"Mozilla/5.0 {platform} {engine} {chrome} {safari} {browser}"

In [107]:
# Пример использования
headers = {
    "User-Agent": random_user_agent(),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
}

### 4.1 Собираем информацию по названию фильма 

#### Проходим по всем фильмам и получаем по несколько фильмов на один поисковый фильм, то есть  
у нас может быть 5 штук Toy Story и он их всех добавит и итоговую таблицу

In [83]:
idx_movie = 15
movie_data.iloc[idx_movie]

item_id                                                              16
movie_title                          French Twist (Gazon maudit) (1995)
release_date                                                01-Jan-1995
video_release_date                                                  NaN
IMDb_url              http://us.imdb.com/M/title-exact?Gazon%20maudi...
unknown                                                               0
action                                                                0
adventure                                                             0
animation                                                             0
children                                                              0
comedy                                                                1
crime                                                                 0
documentary                                                           0
drama                                                           

In [108]:
%%time

# Результат по всем фильмам будем засисывать сюда
all_movie_info = []

for idx_movie in range(739,len(movie_data)): # len(movie_data)

    if idx_movie%100==0:
        rand = random.randint(15,25)
        time.sleep(rand)
    
    rand = random.randint(1,5)
    time.sleep(rand)
    
    # 1. Базовые данные из movie_data
    item_id = movie_data['item_id'].iloc[idx_movie] # - наш внутренний айдишник для нашей задачи
    movie_title = movie_data['movie_title'].iloc[idx_movie]
    
    # 2. Отделяем название от года
    title = movie_data['movie_title'].iloc[idx_movie]
    if len(title) >= 7 and title[-6] == '(' and title[-5:-1].isdigit():
        query_year = int(title[-5:-1])
        query_name = title[:-7]
    else:
        print(query_name)
        # Если не получилось - добавляем пустую запись и идём дальше
        all_movie_info.append([item_id, movie_title] + [None]*12)
        continue

    year = datetime.strptime(movie_data["release_date"].iloc[idx_movie], "%d-%b-%Y").year
    print(idx_movie," ", query_name)

    url = f"https://www.imdb.com/find/?q={query_name.replace(' ', '+')}"

    # Каждые 50 фильмов меняем headers
    if idx_movie%10==0: 
        headers = {"User-Agent": random_user_agent()}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code!=200:
        print(response.status_code)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Находим все результаты поиска
    results = []
    for item in soup.select('li.ipc-metadata-list-summary-item'):
        title = item.select_one('a.ipc-metadata-list-summary-item__t')

        if not title:
            continue
        
        href = title["href"]
        if not href.startswith("/title/"):  # ✅ Фильтруем только фильмы
            continue
        
        result = {
            "title": title.text,
            "url": "https://www.imdb.com" + title["href"],
            "type": item.select_one('div.ipc-metadata-list-summary-item__c').text if item.select_one('div.ipc-metadata-list-summary-item__c') else None,
            "year": item.select_one('span.ipc-metadata-list-summary-item__li').text if item.select_one('span.ipc-metadata-list-summary-item__li') else None,
        }

        results.append(result)


    # Обрабатываем только первый подходящий результат (если есть)
    if results:

        # для каждолго фильма из найденного списка
        for movie in results:
            one_movie_info = []
            
            one_movie_info = [movie_data['item_id'].iloc[idx_movie], movie_data['movie_title'].iloc[idx_movie], movie["url"]]
            one_movie_info.append(movie["title"])


            response = requests.get(movie["url"], headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")

            data_tag = soup.find("script", id="__NEXT_DATA__", type="application/json")
            full_data = json.loads(data_tag.string)
            data = extract_data(full_data)

            one_movie_info = one_movie_info + data

            all_movie_info.append(one_movie_info)

739   Jane Eyre
740   Last Supper, The
741   Ransom
742   Crow: City of Angels, The
743   Michael Collins
744   Ruling Class, The
745   Real Genius
746   Benny & Joon
747   Saint, The
748   MatchMaker, The
749   Amistad
750   Tomorrow Never Dies
751   Replacement Killers, The
752   Burnt By the Sun
753   Red Corner
754   Jumanji
755   Father of the Bride Part II
756   Across the Sea of Time
757   Lawnmower Man 2: Beyond Cyberspace
758   Fair Game
759   Screamers
760   Nick of Time
761   Beautiful Girls
762   Happy Gilmore
763   If Lucy Fell
764   Boomerang
765   Man of the Year
766   Addiction, The
767   Casper
768   Congo
769   Devil in a Blue Dress
770   Johnny Mnemonic
771   Kids
772   Mute Witness
773   Prophecy, The
774   Something to Talk About
775   Three Wishes
776   Castle Freak
777   Don Juan DeMarco
778   Drop Zone
779   Dumb & Dumber
780   French Kiss
781   Little Odessa
782   Milk Money
783   Beyond Bedlam
784   Only You
785   Perez Family, The
786   Roommates
787   Relati

1100   Six Degrees of Separation
1101   Two Much
1102   Trust
1103   C'est arrivé près de chez vous
1104   Firestorm
1105   Newton Boys, The
1106   Beyond Rangoon
1107   Feast of July
1108   Death and the Maiden
1109   Tank Girl
1110   Double Happiness
1111   Cobb
1112   Mrs. Parker and the Vicious Circle
1113   Faithful
1114   Twelfth Night
1115   Mark of Zorro, The
1116   Surviving Picasso
1117   Up in Smoke
1118   Some Kind of Wonderful
1119   I'm Not Rappaport
1120   Umbrellas of Cherbourg, The (Parapluies de Cherbourg, Les)
1121   They Made Me a Criminal
1122   Last Time I Saw Paris, The
1123   Farewell to Arms, A
1124   Innocents, The
1125   Old Man and the Sea, The
1126   Truman Show, The
Truman Show, The
1128   Chungking Express
1129   Jupiter's Wife
1130   Safe
1131   Feeling Minnesota
1132   Escape to Witch Mountain
1133   Get on the Bus
1134   Doors, The
1135   Ghosts of Mississippi
1136   Beautiful Thing
1137   Best Men
1138   Hackers
1139   Road to Wellville, The
1140   Wa

1439   Above the Rim
1440   Moonlight and Valentino
1441   Scarlet Letter, The
1442   8 Seconds
1443   That Darn Cat!
1444   Ladybird Ladybird
1445   Bye Bye, Love
1446   Century
1447   My Favorite Season
1448   Pather Panchali
1449   Golden Earrings
1450   Foreign Correspondent
1451   Lady of Burlesque
1452   Angel on My Shoulder
1453   Angel and the Badman
1454   Outlaw, The
1455   Beat the Devil
1456   Love Is All There Is
1457   Damsel in Distress, A
1458   Madame Butterfly
1459   Sleepover
1460   Here Comes Cookie
1461   Thieves (Voleurs, Les)
1462   Boys, Les
1463   Stars Fell on Henrietta, The
1464   Last Summer in the Hamptons
1465   Margaret's Museum
1466   Saint of Fort Washington, The
1467   Cure, The
1468   Tom and Huck
1469   Gumby: The Movie
1470   Hideaway
1471   Visitors, The (Visiteurs, Les)
1472   Little Princess, The
1473   Nina Takes a Lover
1474   Bhaji on the Beach
1475   Raw Deal
1476   Nightwatch
1477   Dead Presidents
1478   Reckless
1479   Herbie Rides Again
1

**Посмотрим на результат**

In [109]:
columns_final_table = ["item_id",
           "movie_title",
          "url",
          "title",
          "imdb_id",
          "Director",
          "Rating",
           "voteCount",
           "Stars",
          "Budget",
           "Gross worldwide",
           "Runtime",
           "Release day",
            "Release month",
          "Release year"
          ]

print(len(columns_final_table))

15


In [110]:
Final_Table = pd.DataFrame(all_movie_info,columns = columns_final_table)

In [111]:
print(Final_Table.shape)
Final_Table.head(15)

(4489, 15)


,item_id,movie_title,url,title,imdb_id,Director,Rating,voteCount,Stars,Budget,Gross worldwide,Runtime,Release day,Release month,Release year
0,740,Jane Eyre (1996),https://www.imdb.com/title/tt1229822/?ref_=fn_...,Jane Eyre,tt1229822,Cary Joji Fukunaga,7.3,95381.0,Mia Wasikowska,NaN,34710627.0,7200.0,22.0,4.0,2011.0
1,740,Jane Eyre (1996),https://www.imdb.com/title/tt0780362/?ref_=fn_...,Jane Eyre,tt0780362,Ruth Wilson,8.3,23002.0,None,NaN,NaN,3480.0,21.0,1.0,2007.0
2,740,Jane Eyre (1996),https://www.imdb.com/title/tt0116684/?ref_=fn_...,Jane Eyre,tt0116684,Franco Zeffirelli,6.8,10472.0,William Hurt,NaN,5200601.0,6720.0,12.0,4.0,1996.0
3,740,Jane Eyre (1996),https://www.imdb.com/title/tt0085037/?ref_=fn_...,Jane Eyre,tt0085037,Zelah Clarke,8.0,3877.0,None,NaN,NaN,1680.0,9.0,10.0,1983.0
4,740,Jane Eyre (1996),https://www.imdb.com/title/tt0036969/?ref_=fn_...,Jane Eyre,tt0036969,Robert Stevenson,7.5,9904.0,Orson Welles,1705000.0,NaN,5820.0,7.0,4.0,1944.0
5,741,"Last Supper, The (1995)",https://www.imdb.com/title/tt32461003/?ref_=fn...,The Last Supper,tt32461003,Mauro Borrelli,4.9,891.0,James Oliver Wheatley,NaN,6524708.0,6840.0,14.0,3.0,2025.0
6,741,"Last Supper, The (1995)",https://www.imdb.com/title/tt0113613/?ref_=fn_...,Last Supper - Die Henkersmahlzeit,tt0113613,Stacy Title,6.7,16075.0,Cameron Diaz,NaN,459749.0,5520.0,27.0,2.0,1997.0
7,741,"Last Supper, The (1995)",https://www.imdb.com/title/tt5622316/?ref_=fn_...,The Chosen - Der Auserwählte,tt5622316,Dallas Jenkins,9.1,56724.0,None,NaN,180166.0,3600.0,21.0,1.0,2023.0
8,741,"Last Supper, The (1995)",https://www.imdb.com/title/tt28493686/?ref_=fn...,The Last Supper,tt28493686,John Collins,NaN,0.0,None,NaN,NaN,6000.0,NaN,NaN,NaN
9,741,"Last Supper, The (1995)",https://www.imdb.com/title/tt0379497/?ref_=fn_...,Shaam-e-akhar,tt0379497,Fereydoun Jeyrani,6.0,923.0,Katayoun Riahi,NaN,NaN,5760.0,1.0,2.0,2002.0


In [112]:
Final_Table.tail(5)

,item_id,movie_title,url,title,imdb_id,Director,Rating,voteCount,Stars,Budget,Gross worldwide,Runtime,Release day,Release month,Release year
4484,1681,You So Crazy (1994),https://www.imdb.com/title/tt26566918/?ref_=fn...,Why You So Crazy,tt26566918,None,NaN,0.0,None,NaN,NaN,540.0,22.0,4.0,2021.0
4485,1681,You So Crazy (1994),https://www.imdb.com/title/tt13681804/?ref_=fn...,Luis Alvarez you so crazy,tt13681804,Luis Alvarez,NaN,0.0,None,NaN,NaN,NaN,10.0,2.0,2019.0
4486,1681,You So Crazy (1994),https://www.imdb.com/title/tt36294420/?ref_=fn...,"Episode 58: Oh Japan, You So Crazy!",tt36294420,None,NaN,0.0,None,NaN,NaN,3840.0,24.0,11.0,2010.0
4487,1682,Scream of Stone (Schrei aus Stein) (1991),https://www.imdb.com/title/tt1781827/?ref_=fn_...,Hands of Stone,tt1781827,Jonathan Jakubowicz,6.6,18133.0,Edgar Ramírez,20000000.0,4978353.0,6660.0,26.0,8.0,2016.0
4488,1682,Scream of Stone (Schrei aus Stein) (1991),https://www.imdb.com/title/tt0102855/?ref_=fn_...,Cerro Torre: Schrei aus Stein,tt0102855,Werner Herzog,6.2,1550.0,Vittorio Mezzogiorno,6340000.0,NaN,6300.0,3.0,10.0,1991.0


In [113]:
# Сохраняем результат
Final_Table.to_csv(r"4_740_1682_movie_add_big.csv",index=False)

### 4.2 Собираем информацию по ID фильма

Пусть будут следующие нас интересующие фильмы c такими внутренними ID (item_id) из нашей таблицы

In [35]:
a = [16, 256, 529, 904, 958, 1006, 1068, 1128, 1199, 1252, 1412, 1421, 1462]

**Вот их ID на сайте IMDB я нашел сам вручную**

In [114]:
required_List = [
"https://www.imdb.com/title/tt0113149/?",  
"https://www.imdb.com/title/tt0115856/?",  
"https://www.imdb.com/title/tt0089606/?",  
"https://www.imdb.com/title/tt0119590/?",
"https://www.imdb.com/title/tt0090180/?",
"https://www.imdb.com/title/tt0101458/?",
"https://www.imdb.com/title/tt0114808/?",
"https://www.imdb.com/title/tt0113283/?",
"https://www.imdb.com/title/tt0109592/?",
"https://www.imdb.com/title/tt0057345/?",
"https://www.imdb.com/title/tt0113596/?",
"https://www.imdb.com/title/tt0107566/?",
"https://www.imdb.com/title/tt0118100/?"
]

In [115]:
required_List = [[16, 'https://www.imdb.com/title/tt0113149/?'],
 [256, 'https://www.imdb.com/title/tt0115856/?'],
 [529, 'https://www.imdb.com/title/tt0089606/?'],
 [904, 'https://www.imdb.com/title/tt0119590/?'],
 [958, 'https://www.imdb.com/title/tt0090180/?'],
 [1006, 'https://www.imdb.com/title/tt0101458/?'],
 [1068, 'https://www.imdb.com/title/tt0114808/?'],
 [1128, 'https://www.imdb.com/title/tt0113283/?'],
 [1199, 'https://www.imdb.com/title/tt0109592/?'],
 [1252, 'https://www.imdb.com/title/tt0057345/?'],
 [1412, 'https://www.imdb.com/title/tt0113596/?'],
 [1421, 'https://www.imdb.com/title/tt0107566/?'],
 [1462, 'https://www.imdb.com/title/tt0118100/?']]

In [116]:
all_movies = pd.read_csv("usefull_df.csv")

In [117]:
print(all_movies.shape)
all_movies.head()

(1283, 5)


,item_id,movie_title,url,title,imdb_id
0,1,Toy Story (1995),https://www.imdb.com/title/tt0114709/?ref_=fn_...,Toy Story,tt0114709
1,2,GoldenEye (1995),https://www.imdb.com/title/tt0113189/?ref_=fn_...,James Bond 007: GoldenEye,tt0113189
2,3,Four Rooms (1995),https://www.imdb.com/title/tt0113101/?ref_=fn_...,Silvester in fremden Betten,tt0113101
3,4,Get Shorty (1995),https://www.imdb.com/title/tt0113161/?ref_=fn_...,Schnappt Shorty,tt0113161
4,5,Copycat (1995),https://www.imdb.com/title/tt0112722/?ref_=fn_...,Copykill,tt0112722


In [118]:
all_movies.iloc[407]

item_id                                                      416
movie_title                                    Old Yeller (1957)
url            https://www.imdb.com/title/tt0050798/?ref_=fn_...
title                                          Sein Freund Jello
imdb_id                                                tt0050798
Name: 407, dtype: object

In [119]:
%%time

# Результат по всем фильмам будем засисывать сюда 
all_movie_info = []

for i in range(len(required_List)):
    
    idx_movie = movie_data[movie_data['item_id']==required_List[i][0]].index[0]
    
    # idx_movie = i

    print(i, " ",movie_data["movie_title"].iloc[idx_movie])
    
    if idx_movie%150==0:
        rand = random.randint(15,25)
        time.sleep(rand)
    
    rand = random.randint(1,5)
    time.sleep(1)

    # 1. Базовые данные из movie_data
    item_id = movie_data['item_id'].iloc[idx_movie]
    movie_title = movie_data['movie_title'].iloc[idx_movie]

    # если это значение не пустое
    if pd.notna(movie_data["release_date"].iloc[idx_movie]):
        year = datetime.strptime(movie_data["release_date"].iloc[idx_movie], "%d-%b-%Y").year

    # Каждые 50 фильмов меняем headers
    if idx_movie%25==0: 
        headers = {"User-Agent": random_user_agent()}

    one_movie_info = []

    movie["url"] = required_List[i][1]
    # movie["url"] = all_movies["url"].iloc[idx_movie]

    if pd.isna(movie["url"]):
        all_movie_info.append([item_id, movie_title] + [None]*12)
    else:
        one_movie_info = [movie_data['item_id'].iloc[idx_movie], movie_data['movie_title'].iloc[idx_movie], movie["url"]]
        one_movie_info.append(movie["title"])
        response = requests.get(movie["url"], headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        data_tag = soup.find("script", id="__NEXT_DATA__", type="application/json")
        full_data = json.loads(data_tag.string)
        data = extract_data(full_data)

        one_movie_info = one_movie_info + data

        all_movie_info.append(one_movie_info)

0   French Twist (Gazon maudit) (1995)
1   When the Cats Away (Chacun cherche son chat) (1996)
2   My Life as a Dog (Mitt liv som hund) (1985)
3   Ma vie en rose (My Life in Pink) (1997)
4   To Live (Huozhe) (1994)
5   Until the End of the World (Bis ans Ende der Welt) (1991)
6   Star Maker, The (Uomo delle stelle, L') (1995)
7   Heidi Fleiss: Hollywood Madam (1995) 
8   Cemetery Man (Dellamorte Dellamore) (1994)
9   Contempt (Mépris, Le) (1963)
10   Land Before Time III: The Time of the Great Giving (1995) (V)
11   My Crazy Life (Mi vida loca) (1993)
12   Thieves (Voleurs, Les) (1996)
CPU times: total: 1.98 s
Wall time: 48 s


In [120]:
Final_Table = pd.DataFrame(all_movie_info,columns = columns_final_table)

In [121]:
print(Final_Table.shape)
Final_Table.head()

(13, 15)


,item_id,movie_title,url,title,imdb_id,Director,Rating,voteCount,Stars,Budget,Gross worldwide,Runtime,Release day,Release month,Release year
0,16,French Twist (Gazon maudit) (1995),https://www.imdb.com/title/tt0113149/?,Cerro Torre: Schrei aus Stein,tt0113149,Josiane Balasko,6.4,5490,Victoria Abril,NaN,1026646.0,6240,5,10,1995
1,256,When the Cats Away (Chacun cherche son chat) (...,https://www.imdb.com/title/tt0115856/?,Cerro Torre: Schrei aus Stein,tt0115856,Cédric Klapisch,6.9,3673,Garance Clavel,300000.0,1474628.0,5460,14,11,1996
2,529,My Life as a Dog (Mitt liv som hund) (1985),https://www.imdb.com/title/tt0089606/?,Cerro Torre: Schrei aus Stein,tt0089606,Lasse Hallström,7.6,23229,Anton Glanzelius,NaN,8349284.0,6060,15,10,1987
3,904,Ma vie en rose (My Life in Pink) (1997),https://www.imdb.com/title/tt0119590/?,Cerro Torre: Schrei aus Stein,tt0119590,Alain Berliner,7.5,9132,Georges Du Fresne,NaN,2162043.0,5280,6,11,1997
4,958,To Live (Huozhe) (1994),https://www.imdb.com/title/tt0090180/?,Cerro Torre: Schrei aus Stein,tt0090180,William Friedkin,7.3,43233,William Petersen,8000000.0,17311746.0,6960,6,3,1986


In [122]:
print(Final_Table.shape)
Final_Table.tail(3)

(13, 15)


,item_id,movie_title,url,title,imdb_id,Director,Rating,voteCount,Stars,Budget,Gross worldwide,Runtime,Release day,Release month,Release year
10,1412,Land Before Time III: The Time of the Great Gi...,https://www.imdb.com/title/tt0113596/?,Cerro Torre: Schrei aus Stein,tt0113596,Roy Allen Smith,5.6,6838,Scott McAfee,NaN,NaN,4260,15,12,1995
11,1421,My Crazy Life (Mi vida loca) (1993),https://www.imdb.com/title/tt0107566/?,Cerro Torre: Schrei aus Stein,tt0107566,Allison Anders,6.5,2534,Angel Aviles,NaN,3267313.0,5520,15,7,1994
12,1462,"Thieves (Voleurs, Les) (1996)",https://www.imdb.com/title/tt0118100/?,Cerro Torre: Schrei aus Stein,tt0118100,André Téchiné,6.7,2334,Catherine Deneuve,NaN,1018682.0,7020,24,4,1997


In [123]:
# Количество фильмов с путсым значением рейтинга
sum(Final_Table["Rating"].isna())

0

In [124]:
count_non_empty = Final_Table["Budget"].count()
print(f"Количество непустых значений в столбце Budget: {count_non_empty}")

Количество непустых значений в столбце Budget: 5


In [125]:
# Сохраняем результат
Final_Table.to_csv(r"all_movies_addition.csv",index=False)